In [6]:
!pip install agency-swarm gradio requests pydantic


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from agency_swarm import set_openai_key
from getpass import getpass
set_openai_key(getpass("Please enter your openai key: "))

# Sales Manager Agent

In [9]:
salesmgr_instructions = """# SalesManager Agent Instructions

You are an agent that oversees the entire sales process, communicates with other agents, and ensures that the sales data is loaded and analyzed.

### Primary Instructions:
2. Initiate communication with DataAnalystAgent to extract data from Wordpress and perform data analysis on the loaded sales data.
3. Ensure the overall coordination and execution of tasks within the agency.
"""

In [10]:
from agency_swarm import Agent

salesmgr = Agent(name="SalesManager",
            description="Oversees the entire sales process, communicates with other agents, and ensures that the sales data is loaded and analyzed.",
            instructions=salesmgr_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[])

### Custom tools for sales loader agent

## Data Analyst Agent


In [13]:
analyst_instructions = """# DataAnalyst Instructions

You are an agent responsible for extracting, processing and analyzing the sales data. Your role is to use data analysis libraries to generate insights and reports.

### Primary Instructions:
1. Extract the data from Wordpress and prepare it for analysis.
2. Process and analyze the data using data analysis libraries.
3. Generate comprehensive reports and insights.
4. Send the insights and reports to SalesManager for user communication.

"""

### Custom tools for sales agent

In [14]:
from agency_swarm.tools import BaseTool
from pydantic import Field
import pandas as pd

class AnalyzeSalesData(BaseTool):
    """
    This tool should analyze the sales data extracted by the DataExtractorAgent.
    It should identify trends and insights from the sales data, such as sales patterns, high-performing products, and customer behavior.
    The tool should return a structured report with the identified trends and insights.
    """

    analyze: bool = Field(
        ..., description="Yes if the tool should return detailed information about the analysis."
    )

    def run(self):
        """
        The implementation of the run method, where the tool's main functionality is executed.
        This method analyzes the sales data to identify trends and insights, such as sales patterns, high-performing products, and customer behavior.
        It returns a structured report with the identified trends and insights. Currency is assumed to be EUR.
        """
        
        sales_data = self.shared_state.get('sales_data')
        # Convert sales data to a DataFrame for analysis
        df = pd.DataFrame(sales_data)

        # Ensure the DataFrame has the necessary columns
        required_columns = ['Product ID', 'Product Name', 'Quantity', 'Order Date', 'Customer ID', 'Total', 'Net Revenue']
        if not all(column in df.columns for column in required_columns):
            return "The sales data is missing one or more required columns."

        # Convert column types
        df['Order Date'] = pd.to_datetime(df['Order Date'])
        df['Total'] = pd.to_numeric(df['Total'], errors='coerce')
        df['Net Revenue'] = pd.to_numeric(df['Net Revenue'], errors='coerce')
        df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')

        # Analyze sales patterns
        sales_by_date = df.groupby(df['Order Date'].dt.date).agg({'Total': 'sum', 'Net Revenue': 'sum','Quantity': 'sum'}).reset_index()
        sales_trend = sales_by_date.to_dict(orient='records')

        # Identify high-performing products
        top_products = df.groupby(['Product ID', 'Product Name']).agg({'Quantity': 'sum', 'Total': 'sum', 'Net Revenue': 'sum'}).reset_index()
        top_products = top_products.sort_values(by='Total', ascending=False).head(10).to_dict(orient='records')
        
        # Calculate total revenue
        total_income = df['Total'].sum()
        net_revenue = df['Net Revenue'].sum()
        sold_products = df['Quantity'].sum()

        # Analyze customer behavior
        # customer_sales = df.groupby('Customer ID').agg({'Total': 'sum', 'Quantity': 'sum'}).reset_index()
        # customer_behavior = customer_sales.to_dict(orient='records')

        # # Compile the report
        report = {
            "sales_trend": sales_trend,
            "sold_products": sold_products,
            "top_products": top_products,
            "total_income": total_income,
            "net_revenue": net_revenue
        }

        return report

In [24]:
from agency_swarm.tools import BaseTool
from pydantic import Field
import requests
from requests.auth import HTTPBasicAuth
import json
import os

# Define global variables for authentication
site_url = "https://careup.online"
consumer_key = getpass("Please enter your consumer key: ")
consumer_secret = getpass("Please enter your consumer secret: ")

orders_endpoint = f'{site_url}/wp-json/wc/v3/orders'

class ExtractSalesDataFromWordPress(BaseTool):
    """
    This tool should extract sales data from WordPress using the WordPress API.
    It should be able to authenticate with the WordPress site, query the necessary endpoints to retrieve sales data,
    and handle any errors that may occur during the process. The data will be extracted according to a given date range,
    which varies depending on the analysis type. The start date should be set 3 months before the period to be analyzed,
    if the revenue is to be analyzed. The start date should be set to the same day of the first day of the period if the amount of sales is to be analyzed. 
    The extracted data should be returned as a summary of sales, as the data points might be too many. Note that revenue
    is in euros.
    """

    start_date_period: str = Field(
        ..., description="The start date of the period to be analyzed, format: YYYY-MM-DDTHH:MM:SS."
    )
    
    start_date_load: str = Field(
        ..., description="The starting date of the sales to be loaded, format: YYYY-MM-DDTHH:MM:SS. If you want to analyze the revenue for a certain period of time,"
                        +"set the start date 3 months ago. If you want to analyze the amount of sales made for a certain period, set the start date to the same day as the first day of the period."
    )
    
    end_date: str = Field(
        ..., description="The ending date of the sales to be loaded, format: YYYY-MM-DDTHH:MM:SS."
    )
    
    revenue: bool = Field(
        True, description="If you want to analyze the revenue, set this to True. If you want to analyze the amount of sales made, set this to False."
    )

    def run(self):
        """
        The implementation of the run method, where the tool's main functionality is executed.
        This method authenticates with the WordPress site, queries the necessary endpoints to retrieve sales data,
        and handles any errors that may occur during the process.
        """
        
        # Assuming this script is located within the project directory
        base_path = os.getcwd()
        
        # Check if base_path ends with 'data'
        if base_path.endswith('data'):
            base_path = os.path.dirname(base_path)

        # Construct the relative path
        self.shared_state.set('default_folder', base_path)
        self.shared_state.set('data_folder', base_path + '/data')
        os.chdir(self.shared_state.get('data_folder'))
        
        try:
            
            sales_data = []
            page = 1  # Start with the first page

            while True:
                # Parameters including the current page number and date range
                params = {
                    'per_page': 100,  # Number of orders per page (max 100)
                    'status': 'completed',  # Retrieve only completed orders
                    'orderby': 'date',
                    'order': 'desc',
                    'page': page,  # The current page to request
                    'after': self.start_date,  # Filter orders created after this date
                    'before': self.end_date   # Filter orders created before this date
                }

                response = requests.get(orders_endpoint, auth=HTTPBasicAuth(consumer_key, consumer_secret), params=params)
                
                # Get the orders from the response
                orders = response.json()
                
                # Break if no more orders are returned (end of pages)
                if not orders:
                    break
                
                # Process orders on this page
                for order in orders:
                    order_id = order['id']
                    order_date = order['date_created']

                    # Get the date paid and total revenue for each order
                    date_paid = order['date_paid']
                    if self.revenue:
                        if date_paid > self.end_date or date_paid < self.start_date_period or date_paid == None:
                            continue
                    total = order['total']
                    customer_id = order['customer_id']
                    line_items = order['line_items']
                                    
                    for item in line_items:
                        product_id = item['product_id']
                        product_name = item['name']
                        quantity = item['quantity']
                        net_revenue = item['total']
                        
                        # Add the data to the sales_data list
                        sales_data.append({
                            'Order ID': order_id,
                            'Order Date': order_date,
                            'Date Paid': date_paid,
                            'Total': total,
                            'Customer ID': customer_id,
                            'Product ID': product_id,
                            'Product Name': product_name,
                            'Quantity': quantity,
                            'Net Revenue': net_revenue
                        })
                
                # Move to the next page
                page += 1

            # Return the extracted data in a structured format suitable for analysis
            with open("./sales_data.json", "w") as f:
                json.dump(sales_data, f)
            
            self.shared_state.set('sales_data', sales_data)
            os.chdir(base_path)
            return sales_data

        except requests.exceptions.RequestException as e:
            return f"An error occurred while extracting sales data: {e}"

In [25]:
analyzer = Agent(name="DataAnalyst",
            description="Data Analyst is responsible for processing and analyzing the extracted sales data. This agent will use data analysis libraries to generate insights and reports.",
            instructions=analyst_instructions, # can be a file like ./instructions.md
            files_folder=None,
            tools=[ExtractSalesDataFromWordPress, AnalyzeSalesData])


## Setup of agency

In [26]:
agency_manifesto="""# SalesDataAnalyzer Agency Manifesto

The mission of SalesDataAnalyzer is to automate the process of analyzing sales data from Moterik. The agency will extract, process, and interpret sales data to provide actionable insights and reports. The ultimate goal is to enhance decision-making and improve sales strategies based on data-driven insights.

## Goals:
1. Extract sales data from Wordpress efficiently.
2. Analyze and interpret the extracted data to provide meaningful insights.
3. Generate comprehensive reports to aid in decision-making.

## Working Environment:
- The agents will work in a collaborative environment, each focusing on their specialized tasks.
- Communication between agents will be streamlined to ensure smooth data flow and integration.
- The agency will leverage WooCommerce's API for data extraction and Python libraries for data analysis and visualization.

"""

In [27]:
from agency_swarm import Agency

agency = Agency([
    salesmgr,
    [salesmgr, analyzer]
], shared_instructions=agency_manifesto)

Updating agent... DataAnalyst


In [28]:
agency.demo_gradio(height=900)

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


Gradio Blocks instance: 6 backend functions
-------------------------------------------
fn_index=0
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x00000207CAD08C10>
 |-<gradio.components.chatbot.Chatbot object at 0x00000207CAF52DD0>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x00000207CAD08C10>
 |-<gradio.components.chatbot.Chatbot object at 0x00000207CAF52DD0>
fn_index=1
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x00000207CAD08C10>
 |-<gradio.components.chatbot.Chatbot object at 0x00000207CAF52DD0>
 outputs:
 |-<gradio.components.textbox.Textbox object at 0x00000207CAD08C10>
 |-<gradio.components.chatbot.Chatbot object at 0x00000207CAF52DD0>
fn_index=2
 inputs:
 |-<gradio.components.dropdown.Dropdown object at 0x00000207CB045CD0>
 outputs:
fn_index=3
 inputs:
 |-<gradio.templates.Files object at 0x00000207CB06B910>
 outputs:
fn_index=4
 inputs:
 |-<gradio.components.textbox.Textbox object at 0x00000207CAD08C10>
 |-<gradio.components.chatbot.Cha

Message files:  []
Images:  []
THREAD:[ user -> SalesManager ]: URL https://platform.openai.com/playground/assistants?assistant=asst_PQYW2IkiU9T0kolJ2hdcx3RW&mode=assistant&thread=thread_0v08Tr5fCUwIxhmyx428QfS7
THREAD:[ SalesManager -> DataAnalyst ]: URL https://platform.openai.com/playground/assistants?assistant=asst_sZzwBuBLLJS1seqrbgOV4Eht&mode=assistant&thread=thread_9C4z2LVhLl4Bh24xVkdAkpYn
